In [27]:
import pandas as pd 
import numpy as np 
import lightgbm as lgb 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import shap
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../data/model_input.csv')

 Log-Transform Target Variable

In [28]:
# Remove rows where total_charges <= 0
df = df[df['total_charges'] > 0].copy()

# Log-transform the target
df['log_total_charges'] = np.log1p(df['total_charges'])  

 Define Features and Target

In [29]:
feature_cols = [col for col in df.columns if col not in ['total_charges', 'log_charges']]  # adjust if needed
X = df[feature_cols]
y = df['log_total_charges']

Train/Test Split

In [30]:
# Split features and target 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Train LightGBM Model

In [31]:
model = lgb.LGBMRegressor(random_state=42)
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022700 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 854
[LightGBM] [Info] Number of data points in the train set: 1873243, number of used features: 10
[LightGBM] [Info] Start training from score 10.142437


LGBMRegressor(random_state=42)

Predict & Inverse Log Transform

In [32]:
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log) # back to original dollar scale
y_actual = np.expm1(y_test)

Evaluate Model Performance

In [33]:
mae = mean_absolute_error(y_actual, y_pred)
rmse = mean_squared_error(y_actual, y_pred, squared=False)

print("MAE:", mae)
print("RMSE:", rmse)

MAE: 1161.4645394055433
RMSE: 16267.007432697374


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


 Create Residuals

In [34]:
residuals_df = pd.DataFrame({
    'actual': y_actual,
    'predicted': y_pred
})
residuals_df['error'] = residuals_df['actual'] - residuals_df['predicted']
residuals_df['abs_error'] = residuals_df['error'].abs()

 Save Residual Sample

In [35]:
sample_residuals = residuals_df.sample(10000, random_state=42)
sample_residuals.to_csv('../data/sample_residuals_lgbm_10k.csv')

****

In [36]:
# Sample residuals
residuals_df = pd.read_csv('../data/sample_residuals_lgbm_10k.csv', index_col=0)
# Original feautres
model_input_df = pd.read_csv('../data/model_input.csv')

Join on Index to Get All Original Features

In [37]:
# Join on index to get all original features for each residual
merged_df = model_input_df.loc[residuals_df.index].copy()

# Add predictions and errors back in
merged_df["actual"] = residuals_df["actual"]
merged_df["predicted"] = residuals_df["predicted"]
merged_df["error"] = residuals_df["error"]
merged_df["abs_error"] = residuals_df["abs_error"]

In [38]:
merged_df.describe()
merged_df["abs_error"].describe()
merged_df["diagnosis_encoded"].value_counts(normalize=True).head(10)

diagnosis_encoded
137    0.0998
241    0.0478
167    0.0251
150    0.0229
12     0.0228
221    0.0214
67     0.0207
238    0.0206
56     0.0183
177    0.0177
Name: proportion, dtype: float64

Analyze Top 1000 Worst Residuals

In [39]:
top1000 = merged_df.sort_values(by='abs_error', ascending=False).head(1000)
top1000["diagnosis_encoded"].value_counts(normalize=True).head(10)

diagnosis_encoded
241    0.113
7      0.042
8      0.039
63     0.038
69     0.034
112    0.032
137    0.029
67     0.027
249    0.027
238    0.025
Name: proportion, dtype: float64

Top Diagnoses in Full Dataset

In [40]:
model_input_df["diagnosis_encoded"].value_counts(normalize=True).head(10)

diagnosis_encoded
137    0.095809
241    0.047380
167    0.028714
150    0.023119
12     0.021543
67     0.021195
221    0.019503
177    0.018355
238    0.017987
56     0.017978
Name: proportion, dtype: float64

In [41]:
# Load diagnosis map and merge the labels
diagnosis_map = pd.read_csv('../data/diagnosis_mapping.csv')

merged_df = merged_df.merge(diagnosis_map, on='diagnosis_encoded', how='left')
model_input_df = model_input_df.merge(diagnosis_map, on='diagnosis_encoded', how='left')

In [42]:
top_1000 = merged_df.sort_values(by='abs_error', ascending=False).head(1000)

# Worst error diagnoses
top_1000['ccs_diagnosis_description'].value_counts(normalize=True).head(10)

ccs_diagnosis_description
Septicemia (except in labor)                                       0.113
Acute cerebrovascular disease                                      0.042
Acute myocardial infarction                                        0.039
Complication of device; implant or graft                           0.038
Coronary atherosclerosis and other heart disease                   0.034
Heart valve disorders                                              0.032
Liveborn                                                           0.029
Congestive heart failure; nonhypertensive                          0.027
Spondylosis; intervertebral disc disorders; other back problems    0.027
Schizophrenia and other psychotic disorders                        0.025
Name: proportion, dtype: float64

**Septicemia:** Overrepresented in Worst Errors
- It makes up 11.3% of the top 1000 worst predictions

- But only **4.7%** of the total dataset
- More than **2×** its expected frequency
- This suggests the model is consistently underperforming for this diagnosis.

**Liveborn:** Underrepresented in Worst Errors
- Makes up only **2.9%** of top 1000 worst predictions

- But nearly 9.6% of total dataset

- 1/3 of its expected frequency

In [43]:
# Full dataset
model_input_df['ccs_diagnosis_description'].value_counts(normalize=True).head(10)

ccs_diagnosis_description
Liveborn                                                                          0.095809
Septicemia (except in labor)                                                      0.047380
Osteoarthritis                                                                    0.028714
Mood disorders                                                                    0.023119
Alcohol-related disorders                                                         0.021543
Congestive heart failure; nonhypertensive                                         0.021195
Pneumonia (except that caused by tuberculosis or sexually transmitted disease)    0.019503
Other complications of birth; puerperium affecting management of mother           0.018355
Schizophrenia and other psychotic disorders                                       0.017987
Cardiac dysrhythmias                                                              0.017978
Name: proportion, dtype: float64

Generate and analyze SHAP values to explain why the model:
- Overpredicts or underpredicts certain diagnoses (like Septicemia),
- Performs well on others (like Liveborn).




Load the SHAP explainer to analyze feature importance per prediction.

In [44]:
# Create the SHAP explainer using model 
explainer = shap.Explainer(model)

# Apply to the test set only (used for model evaluation)
shap_values = explainer(X_test)

Add Diagnosis Labels to X_test

In [45]:
# Add diagnosis code to the test set for filtering 
X_test_with_diag = X_test.copy()
X_test_with_diag['diagnosis_encoded'] = df.loc[X_test.index, 'diagnosis_encoded']

Filter for Specific Cases

In [46]:
# Septicemia (code 241)
septicemia_cases = X_test_with_diag[X_test_with_diag['diagnosis_encoded'] == 241]

# Liveborn (code 137)
liveborn_cases = X_test_with_diag[X_test_with_diag['diagnosis_encoded'] == 137]

Plot SHAP Summary for Each Diagnosis

In [ ]:
# Septicemia cases — where error is high
shap.plots.bar(shap_values[septicemia_cases.index], max_display=10)

# Liveborn cases — where error is low
shap.plots.bar(shap_values[liveborn_cases.index], max_display=10)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()